# Setup Your Analysis Environment

1) Choose a directory to house your project in: 
```
.../<project-directory>
```


2) Create and activate a fresh Python3 virtual environment (we use Python 3.6.8) there: 
```
$ cd .../<project-directory>
$ python -m virtualenv env 
$ source env/bin/activate
```

3) Download the `cosmicfish` package from Git: 
```
$ git clone git@github.com:ndeporzio/cosmicfish.git
```

4) Install the `cosmicfish` package. Note that its dependencies will install automatically.
```
$ cd cosmicfish
$ pip install . 
```

5) Launch Jupyter and open `tutorial.ipynb` notebook using Jupyter browser
```
$ jupyter notebook
```

6) Create a data folder where the analysis can store spectrum data. This can be anywhere you'd like - you'll specify the path below. 
```
$ mkdir <project-directory>/data
```

7) Install and build CLASS (if you don't already have a build). Note the `cosmicfish` package includes a method for downloading and installing CLASS for you:
```
$ python 
>>> import cosmicfish
>>> cosmicfish.install_class('<project-directory>/class')
>>> exit()
```

# Import & Configure _cosmicfish_

Import the analysis package.

In [ ]:
import cosmicfish as cf 

Import relevant python packages... 

In [ ]:
#Import relevant packages
import os
import numpy as np
import pandas as pd
import seaborn as sns 
import matplotlib.pyplot as plt

Other setup steps... 

In [ ]:
#Instruct pyplot to use seaborn 
sns.set()

Specify the paths from the setup of your analysis environment above.  

In [ ]:
#Set project, data, CLASS directories 
projectdir = cf.correct_path("~/Desktop/cfworkspace/")
datastore = cf.correct_path("~/Desktop/cfworkspace/data.nosync/")
classpath = os.path.join(projectdir, "class")

Specify the granularity of numerical derivatives.

In [ ]:
dstep = 0.008
mu_step = 0.05 #For calculating numerical integral wrt mu between -1 and 1 

# Specify Fiducial Cosmology

In [ ]:
#Setup fiducial cosmology
neutrinofid = {
        "A_s" : 2.2321e-9, 
        "n_s" : 0.96659,
        "omega_b" : 0.02226,
        "omega_cdm" : 0.11271,
        "tau_reio" : 0.059888,
        "h" : 0.70148,
        "T_cmb" : 2.726, # Units [K]
        "N_ncdm" : 3., 
        #"T_ncdm" : 1.5 / 2.726, # Units [K]. Only uncomment when forecasting relics
        "m_ncdm" : 0.02, # Units [eV]
        #"b1L" : 0.7,
        "b0" : 1.0, 
        "alphak2" : 1.0,
        "sigma_fog_0" : 250000 #Units [m s^-2]
        } 

relicfid = {
        "A_s" : 2.2321e-9, 
        "n_s" : 0.96659,
        "omega_b" : 0.02226,
        "omega_cdm" : 0.11271,
        "tau_reio" : 0.059888,
        "h" : 0.70148,
        "T_cmb" : 2.726, # Units [K]
        "N_ncdm" : 1., 
        "T_ncdm" : 0.1, # Units [K]
        "m_ncdm" : 0.02, # Units [eV]
        "b0" : 1.0, 
        "alphak2" : 1.0,
        "sigma_fog_0" : 250000 #Units [m s^-2]
        }

fid = neutrinofid

# Specify Experiment Observational Parameters

In [ ]:
#Setup redshift bins, fiducial relica masses, relic temps to sample
z_table = np.arange(0.65, 1.85, 0.1)
dNdz = np.array([309., 2269., 1923., 2094., 1441., 1353., 1337., 523., 466., 329., 126., 0., 0.])
skycover = 14000. # Sky coverage of survey in degrees^2

# Demonstrate Convergence 

Before running the forecast, we want to ensure our cosmological parameters are well converged about the points we are interested in using to calculate Fisher matrices. To do so, we can use the `convergence` class of `cosmicfish`. 

We pass to `convergence` some fiducial cosmology, and then it will vary the parameters of that fiducial cosmology by step sizes specified by the user and compute the corresponding power spectrum derivatives. 

All 'varyfactors' are computed relative to the fiducial cosmology. That is:

dtheta = varyfactor * theta_fiducial

**WARNING: This process takes a considerable amount of time (~5 mins per varyfactor w/o prexisting data). Only run when you need to.**

In [ ]:
convergencetest = cf.convergence(
    classpath, # Path to CLASS installation
    datastore, # Path to directory holding CLASS output data
    'neutrino', # 'relic' or 'neutrino' forecasting scheme 
    fid, # The fiducial cosmology 
    z_table, # Redshift steps in observation
    dNdz, # Redshift noise in observation
    fcoverage_deg=14000, # Sky coverage in observation
    RSD=True, # Use RSD correction to Pm
    FOG=True, # Use FOG correction to Pm
    AP=True, # Use AP correction to PM
    COV=True, #Use AP Change of Variables correction to PM
    varyfactors=[0.001, 0.002, 0.003] # Relative factors used to compute convergence
    )
    
convergencetest.gen_all_plots() # Display convergence plots

# Run Fisher Forecast

In [ ]:
lightrelic = cf.forecast(
    classpath, 
    datastore, 
    'neutrino', 
    fid, 
    z_table, 
    dNdz, 
    fcoverage_deg=skycover, 
    dstep=0.008,
    RSD=True,
    FOG=True,
    AP=True,
    COV=True)

lightrelic.gen_pm()
lightrelic.gen_fisher(mu_step)

# Add CMB Data

In [ ]:
lightrelic.load_cmb_fisher()

# Export Fisher Matrices

In [ ]:
lightrelic.export_matrices()

# Other Diagnostics

### Print power spectra and related values at benchmark k, mu

In [ ]:
lightrelic.print_v_table(k_index=92)
lightrelic.print_P_table(k_index=92, mu_index=20)

### Compare forecast inferred Pm to CLASS inferred Pm

In [ ]:
#For each z, plot inferred spectrum vs. CLASS data
sns.set()
sns.set_palette("Blues_d", n_colors=2*len(z_table)+1)
plt.figure(figsize=(15, 7.5))
for zidx, zval in enumerate(z_table[0:-1:3]): 
    ps = lightrelic.spectra_mid[zidx]
    plt.semilogx(ps.k_table, lightrelic.Pm[zidx], label=("z = " + str(zval)))
    plt.semilogx(ps.class_pk['k (h/Mpc)']*fid['h'],
             np.array(ps.class_pk['P (Mpc/h)^3'] / np.power(fid['h'],3.)),
             label='CLASS P(k) Data',
             marker='x',
             linestyle=':')
plt.xlabel(r'k [Mpc$^{-1}$]')
plt.ylabel(r'$P_m$ [Mpc$^3$]')
plt.legend()
plt.show()
#Also plot difference between the two 

### Compare forecast corrections with MCMC corrections

In [ ]:
test_k_table = np.geomspace(0.000001, 0.19663801, 100)
gfacts = np.zeros(len(lightrelic.k_table[4]))
ffacts = np.zeros(len(lightrelic.k_table[4]))
for kidx, kval in enumerate(test_k_table):
    gfacts[kidx] = cf.ggrowth(1.05, kval, 0.70148, 0.02226, 0.11271, 0.06/93.14)
    ffacts[kidx] = cf.fgrowth(0.02226, 0.11271, 0.70148, 1.05)

In [ ]:
zindex = 4
muindex = 0

testz = lightrelic.z_steps[zindex]
testmu = lightrelic.mu_table[muindex]

z_str_val = '105'
mu_str_val = '-1'

filestr = 'z_' + z_str_val + '_mu_' + mu_str_val

#Print P_g 
print("P_galaxy at: ")
print("z = ",  lightrelic.z_steps[zindex])
print("mu = ", lightrelic.mu_table[muindex])

linda_k_Pm_relicfast_data = np.loadtxt("/Users/nicholasdeporzio/Documents/Academic/Research/Projects/cosmicfish/P_cb_"+filestr+"_relicfast_Nick_test.dat", skiprows=1, usecols=0, delimiter="\t")
linda_Pm_relicfast_data = np.loadtxt("/Users/nicholasdeporzio/Documents/Academic/Research/Projects/cosmicfish/P_cb_"+filestr+"_relicfast_Nick_test.dat",  skiprows=1, usecols=2, delimiter="\t")

linda_k_Pm_norelicfast_data = np.loadtxt("/Users/nicholasdeporzio/Documents/Academic/Research/Projects/cosmicfish/P_cb_"+filestr+"_no_relicfast_Nick_test.dat",  skiprows=1, usecols=0, delimiter="\t")
linda_Pm_norelicfast_data = np.loadtxt("/Users/nicholasdeporzio/Documents/Academic/Research/Projects/cosmicfish/P_cb_"+filestr+"_no_relicfast_Nick_test.dat",  skiprows=1, usecols=2, delimiter="\t")

linda_k_Pg_relicfast_data = np.loadtxt("/Users/nicholasdeporzio/Documents/Academic/Research/Projects/cosmicfish/P_galaxy_"+filestr+"_relicfast_Nick_test.dat",  skiprows=1, usecols=0, delimiter="\t")
linda_Pg_relicfast_data = np.loadtxt("/Users/nicholasdeporzio/Documents/Academic/Research/Projects/cosmicfish/P_galaxy_"+filestr+"_relicfast_Nick_test.dat",  skiprows=1, usecols=2, delimiter="\t")

linda_k_Pg_norelicfast_data = np.loadtxt("/Users/nicholasdeporzio/Documents/Academic/Research/Projects/cosmicfish/P_galaxy_"+filestr+"_no_relicfast_Nick_test.dat",  skiprows=1, usecols=0, delimiter="\t")
linda_Pg_norelicfast_data = np.loadtxt("/Users/nicholasdeporzio/Documents/Academic/Research/Projects/cosmicfish/P_galaxy_"+filestr+"_no_relicfast_Nick_test.dat",  skiprows=1, usecols=2, delimiter="\t")

linda_k_rsdfog_relicfast_data = np.loadtxt("/Users/nicholasdeporzio/Documents/Academic/Research/Projects/cosmicfish/P_cb_"+filestr+"_relicfast_Nick_test.dat", skiprows=1, usecols=0, delimiter="\t")
linda_rsd_relicfast_data = np.loadtxt("/Users/nicholasdeporzio/Documents/Academic/Research/Projects/cosmicfish/P_cb_"+filestr+"_relicfast_Nick_test.dat",  skiprows=1, usecols=1, delimiter="\t")
linda_fog_relicfast_data = np.loadtxt("/Users/nicholasdeporzio/Documents/Academic/Research/Projects/cosmicfish/P_cb_"+filestr+"_relicfast_Nick_test.dat",  skiprows=1, usecols=3, delimiter="\t")

linda_k_fg_relicfast_data = np.loadtxt("/Users/nicholasdeporzio/Documents/Academic/Research/Projects/cosmicfish/f_and_g_"+filestr+"_relicfast_Nick_test.dat",  skiprows=1, usecols=0, delimiter="\t")
linda_f_relicfast_data = np.loadtxt("/Users/nicholasdeporzio/Documents/Academic/Research/Projects/cosmicfish/f_and_g_"+filestr+"_relicfast_Nick_test.dat",  skiprows=1, usecols=1, delimiter="\t")
linda_g_relicfast_data = np.loadtxt("/Users/nicholasdeporzio/Documents/Academic/Research/Projects/cosmicfish/f_and_g_"+filestr+"_relicfast_Nick_test.dat",  skiprows=1, usecols=2, delimiter="\t")

#Compare Pm plots
sns.set()
sns.set_palette("Blues_d", n_colors=3)
plt.figure(figsize=(15, 7.5))
plt.loglog(np.array(lightrelic.k_table[zindex]), lightrelic.Pm[zindex], label="Nick/Fisher")
plt.loglog(linda_k_Pm_relicfast_data, linda_Pm_relicfast_data, label="Linda/MCMC w/RelicFast")
plt.loglog(linda_k_Pm_norelicfast_data, linda_Pm_norelicfast_data, label="Linda/MCMC wo/RelicFast")
plt.xlabel(r'k [Mpc$^{-1}$]')
plt.ylabel(r'$P_m$ [Mpc$^3$]')
plt.title(r'Pm, z = {0:.2f}, mu = {1:.2f}'.format(testz, testmu))
plt.legend()
plt.show()

#Compare Pg plots
sns.set()
sns.set_palette("Blues_d", n_colors=3)
plt.figure(figsize=(15, 7.5))
plt.loglog(np.array(lightrelic.k_table[zindex]), lightrelic.Pg[zindex, :, muindex], label="Nick/Fisher")
plt.loglog(linda_k_Pg_relicfast_data, linda_Pg_relicfast_data, label="Linda/MCMC w/RelicFast")
plt.loglog(linda_k_Pg_norelicfast_data, linda_Pg_norelicfast_data, label="Linda/MCMC wo/RelicFast")
plt.xlabel(r'k [Mpc$^{-1}$]')
plt.ylabel(r'$P_g$ [Mpc$^3$]')
plt.title(r'Pg, z = {0:.2f}, mu = {1:.2f}'.format(testz, testmu))
plt.legend()
plt.show()

#Also plot difference between the two 
k_vals = np.array(lightrelic.k_table[zindex])
fisher_data_pm = lightrelic.Pm[zindex]
fisher_data_pg = lightrelic.Pg[zindex, :, muindex]
mcmc_data_pm = np.interp(k_vals, linda_k_Pm_relicfast_data, linda_Pm_relicfast_data)
mcmc_data_pg = np.interp(k_vals, linda_k_Pg_relicfast_data, linda_Pg_relicfast_data)

sns.set()
sns.set_palette("Blues_d", n_colors=3)
plt.figure(figsize=(15, 7.5))
plt.semilogx(k_vals, (fisher_data_pm/mcmc_data_pm)-1.0, label="Pm")
plt.semilogx(k_vals, (fisher_data_pg/mcmc_data_pg)-1.0, label="Pg")
plt.title(r'(Fisher/MCMC) - 1, z = {0:.2f}, mu = {1:.2f}'.format(testz, testmu))
plt.xlabel(r'k [Mpc$^{-1}$]')
plt.ylabel(r'Ratio')
plt.legend()
plt.show()


#Compare RSD Corrections
sns.set()
sns.set_palette("Blues_d", n_colors=3)
plt.figure(figsize=(15, 7.5))
plt.semilogx(np.array(lightrelic.k_table[zindex]), lightrelic.RSD[zindex, :, muindex], label="Nick/Fisher")
plt.semilogx(linda_k_rsdfog_relicfast_data, linda_rsd_relicfast_data, label="Linda/MCMC w/RelicFast")
plt.title(r'RSD Comparison, z = {0:.2f}, mu = {1:.2f}'.format(testz, testmu))
plt.xlabel(r'k [Mpc$^{-1}$]')
plt.ylabel(r'RSD')
plt.legend()
plt.show()

#Compare FOG corrections
sns.set()
sns.set_palette("Blues_d", n_colors=3)
plt.figure(figsize=(15, 7.5))
plt.semilogx(np.array(lightrelic.k_table[zindex]), lightrelic.FOG[zindex, :, muindex], label="Nick/Fisher")
plt.semilogx(linda_k_rsdfog_relicfast_data, linda_fog_relicfast_data, label="Linda/MCMC w/RelicFast")
plt.title(r'FOG Comparison, z = {0:.2f}, mu = {1:.2f}'.format(testz, testmu))
plt.xlabel(r'k [Mpc$^{-1}$]')
plt.ylabel(r'FOG')
plt.legend()
plt.show()

#Compare g growth factor
sns.set()
sns.set_palette("Blues_d", n_colors=2)
plt.figure(figsize=(15, 7.5))
plt.semilogx(linda_k_fg_relicfast_data, linda_g_relicfast_data, label="MCMC")
plt.semilogx(test_k_table, gfacts, label="Fisher")
plt.xlabel(r'k [Mpc$^{-1}$]')
plt.ylabel(r'g')
plt.title(r'g Comparison, z = {0:.2f}, mu = {1:.2f}'.format(testz, testmu))
plt.legend()
plt.show()

#Compare f growth factor
sns.set()
sns.set_palette("Blues_d", n_colors=2)
plt.figure(figsize=(15, 7.5))
plt.semilogx(linda_k_fg_relicfast_data, linda_f_relicfast_data, label="MCMC")
plt.semilogx(test_k_table, ffacts, label="Fisher")
plt.xlabel(r'k [Mpc$^{-1}$]')
plt.ylabel(r'f')
plt.title(r'f Comparison, z = {0:.2f}, mu = {1:.2f}'.format(testz, testmu))
plt.legend()
plt.show()